# Query NFL Team Article Database and Run Queries

In [1]:
# Packages for NFL Database

import psycopg2
import config as config
import teamQuery as tq

In [2]:
len(tq.teamQueryCounts("Falcons"))

286

In [3]:
def quick_team_corpus(team):
    return [text[3] for text in tq.teamQueryTexts(team)]

In [4]:
red = quick_team_corpus("Redskins")

In [5]:
len(red)

411

In [6]:
# Packages for LDA

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from time import time
import nltk
from nltk.stem import SnowballStemmer, WordNetLemmatizer


In [7]:
# Initializations

nltk.download('wordnet')

n_features = 1000
n_topics = 10
n_top_words = 5

base_tokenizer = CountVectorizer().build_tokenizer()
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

tokenize = None
#tokenize = lambda text: [stemmer.stem(item) for item in base_tokenizer(text)]
#tokenize = lambda text: [lemmatizer.lemmatize(item) for item in base_tokenizer(text)]


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Anthony\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
# The print top words function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message+"\n")
    print()
        

In [9]:
# Get the Samples for a team
team = "Ravens"
samples = quick_team_corpus(team)

In [10]:
# TF-IDF Vectorization With Prepocessing
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=.95,
                                   min_df=2,
                                   max_features=n_features,
                                   ngram_range=(1,3),
                                   tokenizer=tokenize,
                                   stop_words='english')

t0 = time()
tfidf = tfidf_vectorizer.fit_transform(samples)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 1.636s.


In [11]:
# NMF Model Fit
print("Fitting the NMF model with tf-idf freatures,  "
      'n_samples=%d and n_features=%d...'
      % (len(samples), n_features))

t0 = time()

nmf = NMF(n_components=n_topics,
          random_state=1,
          alpha=.1,
          l1_ratio=.5)

nmfResult = nmf.fit(tfidf)
print("done in %0.3fs." % (time() - t0))

tfidf_feature_names = tfidf_vectorizer.get_feature_names()

Fitting the NMF model with tf-idf freatures,  n_samples=315 and n_features=1000...
done in 0.526s.


In [12]:
# Show the Topics
print("Topics in NMF model:")
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topics in NMF model:
Topic #0: game, just, defense, going, good

Topic #1: williams, practice, injury, defensive, smith

Topic #2: preseason, woodrum, ricard, roster, bradley

Topic #3: langford, practice squad, squad, woodhead, practice

Topic #4: london, fans, fan, nfl, trip

Topic #5: purple, pm, fans, mt, mt bank

Topic #6: wallace, perriman, maclin, yards, wide

Topic #7: bergstrom, yanda, guard, offensive, traded

Topic #8: collins, school, yards, carries, carl

Topic #9: flacco, flacco said, said, joe, offense




In [13]:
# TF Vectorization - raw term counts with Preprocessing
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=.95,
                                min_df=2,
                                ngram_range=(1,3),
                                max_features=n_features,
                                tokenizer=tokenize,
                                stop_words='english')

t0 = time()
tf = tf_vectorizer.fit_transform(samples)
print('done in %0.3fs.' % (time() - t0))

Extracting tf features for LDA...
done in 1.632s.


In [14]:
print("Fitting the LDA model  with tf freatures,  "
      'n_samples=%d and n_features=%d...'
      % (len(samples), n_features))

t0 = time()

lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0,
                                )

lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names()

Fitting the LDA model  with tf freatures,  n_samples=315 and n_features=1000...
done in 2.723s.


In [15]:
print("Topics in LDA model:")
print_top_words(lda, tf_feature_names, n_top_words)

Topics in LDA model:
Topic #0: game, like, just, said, think

Topic #1: injury, flacco, baltimore, team, coach

Topic #2: injury, practice, williams, week, game

Topic #3: hurst, guard, tackle, left, said

Topic #4: baltimore, school, community, game, city

Topic #5: purple, fans, cancer, stadium, game

Topic #6: going, play, said, good, just

Topic #7: preseason, woodrum, team, game, nfl

Topic #8: foster, preseason, said, game, football

Topic #9: game, just, said, going, play




# Visualization of Topic Models

In [16]:
lda

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [17]:
nmf

NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200,
  n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [18]:
import pyLDAvis.sklearn

In [19]:
pyLDAvis.enable_notebook()

In [20]:
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

c:\users\anthony\scripts\nfl-topics\venv\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
9      72.224402        1       1 -0.020722 -0.061154
2      20.238952        1       2 -0.113843  0.036808
7       4.097299        1       3 -0.009519 -0.040206
5       1.670474        1       4  0.090202  0.024736
3       0.991276        1       5 -0.012560  0.006977
4       0.569385        1       6  0.037014  0.003779
8       0.108327        1       7  0.005812  0.010321
0       0.033296        1       8  0.004793 -0.002624
6       0.033295        1       9  0.008953  0.001518
1       0.033295        1      10  0.009870  0.019845, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
309   Default  1449.000000           game  1449.000000  30.0000  30.0000
429   Default  1023.000000           just  1023.000000  29.0000  29.0000
760   Default   967.000000           said   967.000000  28.0000  28.0000
72    Default   571.000000      baltimore   571.000000  27.0000  27.0000
322   Default   682.000000          going   682.000000  26.0000  26.0000
629   Default   819.000000           play   819.000000  25.0000  25.0000
324   Default   635.000000           good   635.000000  24.0000  24.0000
467   Default   539.000000           like   539.000000  23.0000  23.0000
952   Default   852.000000           week   852.000000  22.0000  22.0000
288   Default   578.000000         flacco   578.000000  21.0000  21.0000
863   Default   546.000000           team   546.000000  20.0000  20.0000
208   Default   562.000000        defense   562.000000  19.0000  19.0000
362   Default   550.000000            hes   550.000000  18.0000  18.0000
773   Default   626.000000         season   626.000000  17.0000  17.0000
880   Default   401.000000          think   401.000000  16.0000  16.0000
574   Default   511.000000        offense   511.000000  15.0000  15.0000
649   Default   496.000000       practice   496.000000  14.0000  14.0000
894   Default   502.000000           time   502.000000  13.0000  13.0000
703   Default   374.000000         really   374.000000  12.0000  12.0000
71    Default   405.000000           ball   405.000000  11.0000  11.0000
336   Default   343.000000           guys   343.000000  10.0000  10.0000
393   Default   481.000000         injury   481.000000   9.0000   9.0000
295   Default   354.000000       football   354.000000   8.0000   8.0000
312   Default   528.000000          games   528.000000   7.0000   7.0000
441   Default   360.000000           know   360.000000   6.0000   6.0000
969   Default   361.000000            win   361.000000   5.0000   5.0000
872   Default   383.000000          thats   383.000000   4.0000   4.0000
991   Default   431.000000           year   431.000000   3.0000   3.0000
346   Default   424.000000       harbaugh   424.000000   2.0000   2.0000
500   Default   382.000000           make   382.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
945   Topic10     0.053977         watson   105.003844   0.4343  -6.3569
584   Topic10     0.043966         opener    63.754899   0.7281  -6.5620
716   Topic10     0.045296         report    70.574457   0.6563  -6.5322
718   Topic10     0.041520      reporters    57.059924   0.7818  -6.6193
975   Topic10     0.049891       woodhead   105.759618   0.3484  -6.4356
393   Topic10     0.070014         injury   481.671835  -0.8288  -6.0968
195   Topic10     0.041686          danny    70.588191   0.5731  -6.6153
160   Topic10     0.057788          coach   264.817304  -0.4225  -6.2887
288   Topic10     0.069616         flacco   578.499996  -1.0177  -6.1025
72    Topic10     0.067934      baltimore   571.810732  -1.0305  -6.1269
863   Topic10     0.066015           team   546.208692  -1.0133  -6.1556
879   Topic10     0.053835         things   250.701830  -0.4386  -6.3595
723   Topic10     0.0508